In [1]:
%cd ../..

/home/ivanstefanov/Repositories/used-cars-price-prediction


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split

In [11]:
from price_predictor.data_preparation.feature_engineering import (
    add_country_feature,
    add_class_feature
)
from price_predictor.data_preparation.pick_or_drop_features import (
    pick_by_datatypes,
    drop_anonymous_features,
    drop_categorical_features
)
from price_predictor.trainers.train_ensamble_model import train_random_forest
from price_predictor.evaluate import evaluate_model

In [6]:
data = pd.read_csv('data/cars.csv')
data.dropna(inplace=True)

## Try numerical with car country feature 

In [7]:
df = add_country_feature(data)

df = drop_anonymous_features(df)
df = drop_categorical_features(df)
df

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,price_usd,is_exchangeable,number_of_photos,up_counter,duration_listed,country_of_origin
0,190000,2010,False,2.5,False,10900.00,False,9,13,16,Japan
1,290000,2002,False,3.0,False,5000.00,True,12,54,83,Japan
2,402000,2001,False,2.5,False,2800.00,True,4,72,151,Japan
3,10000,1999,False,3.0,False,9999.00,True,9,42,86,Japan
4,280000,2001,False,2.5,False,2134.11,True,14,7,7,Japan
...,...,...,...,...,...,...,...,...,...,...,...
38526,290000,2000,False,3.5,False,2750.00,True,5,85,301,United States
38527,321000,2004,False,2.2,False,4800.00,True,4,20,317,United States
38528,777957,2000,False,3.5,False,4300.00,False,3,63,369,United States
38529,20000,2001,False,2.0,False,4000.00,True,7,156,490,United States


In [8]:
dummies = pd.get_dummies(df)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(dummies.drop('price_usd', axis=1), dummies['price_usd'])

## Random Forest

In [10]:
rf = train_random_forest(X_train, y_train)
evaluate_model(rf, X_test, y_test)

{'rmse': 2311.622751146995,
 'mae': 1376.8672132201666,
 'r2': 0.8712026001232113}

## Nonsuprisingly, it boosted the performance of our model
## The country feature is correlated with the car make, but lets try if both would be better

In [12]:
df = add_country_feature(data)
df = add_class_feature(data)

df = drop_anonymous_features(df)
df = drop_categorical_features(df)
df

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,price_usd,is_exchangeable,number_of_photos,up_counter,duration_listed,country_of_origin,class
0,190000,2010,False,2.5,False,10900.00,False,9,13,16,Japan,mid-end
1,290000,2002,False,3.0,False,5000.00,True,12,54,83,Japan,mid-end
2,402000,2001,False,2.5,False,2800.00,True,4,72,151,Japan,mid-end
3,10000,1999,False,3.0,False,9999.00,True,9,42,86,Japan,mid-end
4,280000,2001,False,2.5,False,2134.11,True,14,7,7,Japan,mid-end
...,...,...,...,...,...,...,...,...,...,...,...,...
38526,290000,2000,False,3.5,False,2750.00,True,5,85,301,United States,mid-end
38527,321000,2004,False,2.2,False,4800.00,True,4,20,317,United States,mid-end
38528,777957,2000,False,3.5,False,4300.00,False,3,63,369,United States,mid-end
38529,20000,2001,False,2.0,False,4000.00,True,7,156,490,United States,mid-end


In [13]:
dummies = pd.get_dummies(df)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(dummies.drop('price_usd', axis=1), dummies['price_usd'])

## Random Forest

In [15]:
rf = train_random_forest(X_train, y_train)
evaluate_model(rf, X_test, y_test)

{'rmse': 2156.4583032123737,
 'mae': 1312.7871045843892,
 'r2': 0.8856946621571082}

### A little better than without it

## Conclusion: the car country feature did give us results but still cannot beat the brute force approach of using all the data given to us